In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
import ipywidgets as widgets
from rt2_assignment1.srv import Command
from IPython.display import display
from tf import transformations
import math

rospy.init_node('robot_control_node')

# start/stop the robot
## user interface
- starting/ stopping the robot "random position" behaviour by using two buttons
- send the service /user_interface when the user push the button

In [2]:
ui_client = rospy.ServiceProxy('/user_interface', Command)


start_button = Button(description='start', layout=Layout(width='auto', align="center", grid_area='b1'), style=ButtonStyle(button_color='lightblue')) 


stop_button = Button(description='stop', layout=Layout(width='auto', grid_area='b2'), style=ButtonStyle(button_color='moccasin')) 

output = widgets.Output() 
moving_random_pos=0

def click_start(change):
    global moving_random_pos
    with output:
        if moving_random_pos==0:
            ui_client("start")
            print('the robot starts to move to the random position')
            moving_random_pos=1
        else:
            print('the robot is going to the previous random position')
        
def click_stop(change):
    global moving_random_pos
    with output:
        print('stop')
        if moving_random_pos==1:
            ui_client("stop")
            print('click the stop button')
        moving_random_pos=0

start_button.on_click(click_start)
stop_button.on_click(click_stop)

HBox([start_button,stop_button])

# set the cmd_vel
- setting the linear and angular velocity by using two Sliders
- send /cmd_vel the value of two sliders

In [3]:
msg = Twist()

a = widgets.FloatSlider(
    min=-1.0,
    max=1.0,
    step=0.1,
    description='linear velocity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)

display(a)

b = widgets.FloatSlider(
    min=-1.0,
    max=1.0,
    step=0.1,
    description='angular velocity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)

display(b)

pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

def linear_vel_change(change):
    global msg,moving_random_pos
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x = change['new']
    pub.publish(msg)
    
def angular_vel_change(change):
    global msg,moving_random_pos
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.angular.z = change['new']
    pub.publish(msg)

a.observe(linear_vel_change, names='value')
b.observe(angular_vel_change, names='value')

FloatSlider(value=0.0, continuous_update=False, description='linear velocity:', max=1.0, min=-1.0, readout_for…

FloatSlider(value=0.0, continuous_update=False, description='angular velocity:', max=1.0, min=-1.0, readout_fo…

#  robot controller
- directly controlling the robot movements by using 5 Buttons, (forward, turn right, backward, turn left, stop), possibly placed in an intuitive fashion
- depending on the button, send the /cmd_vel to the robot

In [4]:
left_button = Button(description='←', layout=Layout(width='auto', grid_area='left'), style=ButtonStyle(button_color='salmon')) 
up_button = Button(description='↑',layout=Layout(width='auto', grid_area='up'), style=ButtonStyle(button_color='salmon')) 
right_button = Button(description='→', layout=Layout(width='auto', grid_area='right'), style=ButtonStyle(button_color='salmon')) 
down_button = Button(description='↓', layout=Layout(width='auto', grid_area='down'), style=ButtonStyle(button_color='salmon')) 

def click_up(change):
    global moving_random_pos,msg
    print("Go straight")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x=0.5
    msg.linear.y=0.0
    msg.angular.z=0.0
    pub.publish(msg)
        
def click_down(change):
    global moving_random_pos,msg
    print("Go back")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x=-0.5
    msg.linear.y=0.0
    msg.angular.z=0.0
    pub.publish(msg)

def click_right(change):
    global moving_random_pos,msg
    print("Go right")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x=0.1
    msg.linear.y=0.0
    msg.angular.z=0.5
    pub.publish(msg)

def click_left(change):
    global moving_random_pos,msg
    print("Go left")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
    moving_random_pos=0
    msg.linear.x=0.1
    msg.linear.y=0.0
    msg.angular.z=-0.5
    pub.publish(msg)
            
up_button.on_click(click_up)
down_button.on_click(click_down)
right_button.on_click(click_right)
left_button.on_click(click_left)

GridBox(children=[left_button,down_button,right_button, up_button], 
        layout=Layout( 
            width='20%', 
            grid_template_rows='auto auto', 
            grid_template_columns='33% 33% 33%', 
            grid_template_areas=''' 
            "left up right" 
            ". down . " 
            ''') 
       ) 

GridBox(children=(Button(description='←', layout=Layout(grid_area='left', width='auto'), style=ButtonStyle(but…

Go straight
Go straight
Go straight
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go back
the robot is going to the previous random position and the goal is canceled
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
Go back
